In [1]:
import pandas as pd
import numpy as np
import os
import requests
import datetime
import pytz
import grequests
import time

In [21]:
HAMO_URL = 'https://sp-ride.tp-tsc.com/omsp/external'
APIKEY = os.getenv('HAMO_APIKEY')

# get stations, one call
def get_available_stations():
    '''
    returns available stations
    '''
    # ping tmc
    querystring = {"zone_id":"1","lang":"en"}
    
    headers = {
        'omms_auth': APIKEY
        }
    url = HAMO_URL+"/station/list"
    response = requests.request("GET", url, headers=headers, params=querystring)
    stations = []
    for  area in response.json()['zone']['areas']:
        for station in  area['stations']:
            stations.append(station)    
    # return list of stations that are open 
    return stations

# get station states
def get_station_state(station_ids, parallel_req = False):
    '''
    returns idle vehicles at each station in a dataframe
    '''
    idle_vehicles = np.zeros(len(station_ids))
    available_parking = np.zeros(len(station_ids))
    is_open = np.zeros(len(station_ids))
    headers = {
        'omms_auth': APIKEY
    }
    url = HAMO_URL+"/station/detail"
    reqs = []
    t1 = time.time()
    if parallel_req:
        for i, s_id in enumerate(station_ids):
            #print "{}/{}, id: {}".format(i,len(station_ids),s_id)
            querystring = {"zone_id":"1","station_id":str(s_id),"lang":"en"}
            # response = requests.request("GET", url, headers=headers, params=querystring)
            reqs.append(grequests.get(url, headers=headers, params=querystring))
        print "sending requests"
        responses = grequests.map(reqs)
        for i,response in enumerate(responses):
            station = response.json()['station']
            idle_vehicles[i] = station['available_car']
            available_parking[i] = station['parking_space_free']
            is_open[i] = station['opening_flag_today']
    else:
        for i, s_id in enumerate(station_ids):
            querystring = {"zone_id":"1","station_id":str(s_id),"lang":"en"}
            response = requests.request("GET", url, headers=headers, params=querystring)
            station = response.json()['station']
            idle_vehicles[i] = station['available_car']
            available_parking[i] = station['parking_space_free']
            is_open[i] = station['opening_flag_today']
    print "Time of requests: {}".format(time.time() - t1)
    stations_state = pd.DataFrame(index= station_ids)
    stations_state['idle_vehicles'] = idle_vehicles
    stations_state['available_parking'] = available_parking
    stations_state['is_open'] = is_open
    return stations_state

In [12]:
stations = pd.DataFrame(get_available_stations())

Get stations via sequential method:

In [22]:
stations_state = get_station_state(stations['station_id'], parallel_req = False)

Time of requests: 146.028296947


Get stations via parallel method:

In [23]:
stations_state = get_station_state(stations['station_id'], parallel_req = True)

sending requests
Time of requests: 44.5434892178
